<a href="https://colab.research.google.com/github/eyaJLIDI/churn_prediction_NL/blob/main/shiftData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 20.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=e56c3dde81f7762d322b25c763b60cb5488ef28e271c5d28fa79357f97a3cec7
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import *
from pyspark.sql.functions import monotonically_increasing_id 
from pyspark.sql import functions as F
from pyspark.sql import Window
import warnings
warnings.simplefilter(action='ignore')
from textwrap import wrap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth', 800)
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from pyspark.ml.classification import GBTClassifier, RandomForestClassifier ,  DecisionTreeClassifier , LinearSVC
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler

# create a Spark session
spark =  SparkSession \
        .builder \
        .appName("NL+ churn prediction") \
        .getOrCreate()

In [3]:
def shift_data(df_data):
  import pyspark.pandas as ps
  df = df_data.toPandas()
  data = df[['CLT_REF','nombre']].groupby('CLT_REF').mean()
  data = data.reset_index()
  data.rename(columns = {'nombre': 'trajets_moyen'}, inplace = True)
  data = data.merge(df, on='CLT_REF')
  data['evol_trajet'] = 0
  data.loc[(data['nombre'] > data['trajets_moyen']*2 ) | (data['nombre'] < data['trajets_moyen']*0.5), "evol_trajet"] = 1
  data[data['evol_trajet'] == 1]
  data.drop(columns=['trajets_moyen'],inplace=True)
  df_temps = data[['CLT_REF','CHURN','date']].groupby(['CLT_REF'])['CHURN','date'].shift(-1)
  df_temps.rename(columns={'CHURN':'CHURN_shift','date':'date_shift'},inplace=True)
  data = df_temps.join(data)
  data
  data.drop(columns=['CHURN','date','date_shift','CLT_REF','card_number'],inplace=True)
  data.columns
  data.dropna(subset=['CHURN_shift'],inplace=True)
  data.fillna(method ='pad',inplace=True)
  data.fillna(method ='bfill',inplace=True)
  data
  return data